In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import util
from scipy import stats

In [3]:
pd.set_option('max_columns', None)

In [2]:
ksi_df = pd.read_csv('Data/KSI.csv')
road_merge = pd.read_csv('Data/KSI_Road_Hazardous.csv')

In [5]:
ksi_df

,X,Y,INDEX_,ACCNUM,YEAR,DATE,TIME,HOUR,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,WARDNUM,DIVISION,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,POLICE_DIVISION,HOOD_ID,NEIGHBOURHOOD,ObjectId
0,-8.844611e+06,5.412414e+06,3387730,892658,2006,2006/03/11 05:00:00+00,852,8,BLOOR ST W,DUNDAS ST W,<Null>,Major Arterial,Toronto and East York,4,11,43.656345,-79.452490,Intersection,At Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Driver,unknown,None,<Null>,South,"Automobile, Station Wagon",Turning Left,Failed to Yield Right of Way,Unknown,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,<Null>,<Null>,D11,88,High Park North (88),1
1,-8.844611e+06,5.412414e+06,3387731,892658,2006,2006/03/11 05:00:00+00,852,8,BLOOR ST W,DUNDAS ST W,<Null>,Major Arterial,Toronto and East York,4,11,43.656345,-79.452490,Intersection,At Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Pedestrian,65 to 69,Fatal,<Null>,North,Other,<Null>,<Null>,<Null>,Vehicle turns left while ped crosses with ROW ...,Crossing with right of way,Unknown,<Null>,<Null>,<Null>,Yes,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,<Null>,<Null>,D11,88,High Park North (88),2
2,-8.816480e+06,5.434843e+06,3388101,892810,2006,2006/03/11 05:00:00+00,915,9,MORNINGSIDE AVE,SHEPPARD AVE E,<Null>,Major Arterial,Scarborough,25,42,43.801943,-79.199786,Intersection,At Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Turning Movement,Motorcycle Driver,45 to 49,Fatal,<Null>,East,Motorcycle,Turning Right,Disobeyed Traffic Control,Unknown,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,Yes,<Null>,<Null>,D42,131,Rouge (131),3
3,-8.816480e+06,5.434843e+06,3388102,892810,2006,2006/03/11 05:00:00+00,915,9,MORNINGSIDE AVE,SHEPPARD AVE E,<Null>,Major Arterial,Scarborough,25,42,43.801943,-79.199786,Intersection,At Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Turning Movement,Driver,unknown,None,<Null>,South,"Automobile, Station Wagon",Going Ahead,Driving Properly,Unknown,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,Yes,<Null>,<Null>,D42,131,Rouge (131),4
4,-8.822759e+06,5.424516e+06,3387793,892682,2006,2006/03/12 05:00:00+00,240,2,EGLINTON AVE E,COMMONWEALTH AVE,<Null>,Major Arterial,Scarborough,"21,20",41,43.734945,-79.256190,Mid-Block,<Null>,No Control,Clear,Dark,Dry,Fatal,Pedestrian Collisions,Driver,25 to 29,None,<Null>,West,"Automobile, Station Wagon",Going Ahead,Other,"Ability Impaired, Alcohol",<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,D41,138,Eglinton East (138),5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16855,-8.820837e+06,5.421411e+06,81509784,1636966,2020,2020/08/30 04:00:00+00,1340,13,BRIMLEY RD,BARKDENE HILLS,50 m South of,<Null>,<Null>,<Null>,<Null>,43.714793,-79.238926,Mid-Block,Non Intersection,No Control,Clear,Daylight,Dry,Non-Fatal Injury,Cyclist Collisions,Cyclist,25 to 29,Major,<Null>,North,Bicycle,Going Ahead,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Driving Properly,Normal,<Null>,Yes,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,<Null>,<Null>,D41,123,Cliffcrest (123),16856
16856,-8.820068e+06,5.425334e+06,81505452,1650701,2020,2020/09/01 04:00:00+00,1205,12,EGLINTON AVE E,BELLAMY RD N,<Null>,Major Arterial,Scarborough,"21,20",43,43.740256,-79.232021,Intersection,At Inter

In [4]:
ksi_df = ksi_df.iloc[ksi_df['ACCNUM'].drop_duplicates().index]

In [5]:
ksi_df['LATITUDE'] = ksi_df['LATITUDE'].apply(util.truncate, args = (2, ))
ksi_df['LONGITUDE'] = ksi_df['LONGITUDE'].apply(util.truncate, args = (2, ))

In [6]:
# dont run this, it just makes life harder

#ksi_df.replace('<Null>', np.nan, inplace = True)

In [6]:
# its ugly, its stupid, buts its mine, and it works

l = []
for col in ['INVAGE', 'WARDNUM', 'RDSFCOND', 'VISIBILITY', 'LIGHT', 'TRAFFCTL', 'VEHTYPE', 'DRIVACT', 'DRIVCOND', 'SPEEDING', 'AG_DRIV', 'ALCOHOL', 'DISABILITY', 'REDLIGHT']:    
    l.append(ksi_df.groupby(['LATITUDE', 'LONGITUDE'])[col].apply(lambda x: x.mode().iloc[0]).reset_index())

In [ ]:
# include average of hour
# include age - but need to do feature engineering

In [7]:
# who cares about vectorization?
# loop like a neanderthal

looper = l[0]
for df in range(1, len(l)):
    looper = looper.merge(l[df], on = ['LATITUDE', 'LONGITUDE'], how = 'inner')

In [8]:
ksi_df[(ksi_df['LATITUDE'] == 43.59) & (ksi_df['LONGITUDE'] == -79.54)]

,X,Y,INDEX_,ACCNUM,YEAR,DATE,TIME,HOUR,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,WARDNUM,DIVISION,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,POLICE_DIVISION,HOOD_ID,NEIGHBOURHOOD,ObjectId
4311,-8.854552e+06,5.402603e+06,5444973,1075972,2008,2008/11/24 05:00:00+00,46,0,LAKE SHORE Blvd W,FORTIETH St,<Null>,Major Arterial,Etobicoke York,3,22,43.59,-79.54,Intersection,<Null>,No Control,Clear,Dark,Dry,Non-Fatal Injury,Turning Movement,Driver,15 to 19,Major,<Null>,West,"Automobile, Station Wagon",Making U Turn,Improper Turn,Normal,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,D22,19,Long Branch (19),4312
12779,-8.854526e+06,5.402679e+06,80500048,6000042274,2016,2016/01/08 05:00:00+00,520,5,BROWN S LINE,LAKE SHORE BLVD W,30 m North of,Major Arterial,Etobicoke York,3,22,43.59,-79.54,Mid-Block,Non Intersection,No Control,Clear,"Dark, artificial",Wet,Fatal,SMV Other,Driver,40 to 44,Fatal,2,South,"Automobile, Station Wagon",Going Ahead,Speed too Fast For Condition,Unknown,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,Yes,<Null>,<Null>,<Null>,D22,19,Long Branch (19),12780
12955,-8.854810e+06,5.403630e+06,80719030,6001650762,2016,2016/09/16 04:00:00+00,1955,19,BROWNS LINE,WOODBURY RD,12 m East of,Major Arterial,Etobicoke York,3,22,43.59,-79.54,Mid-Block,At Intersection,Pedestrian Crossover,Clear,Dark,Dry,Non-Fatal Injury,Pedestrian Collisions,Driver,20 to 24,None,<Null>,North,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,D22,20,Alderwood (20),12956
13662,-8.854584e+06,5.402622e+06,80609330,7000125598,2017,2017/01/21 05:00:00+00,40,0,BROWNS LINE,LAKESHORE W BROWNS LI RAMP,20 m North,Major Arterial,Etobicoke York,3,22,43.59,-79.54,Mid-Block,Non Intersection,No Control,"Fog, Mist, Smoke, Dust","Dark, artificial",Wet,Fatal,SMV Other,Driver,20 to 24,Fatal,3,South,Delivery Van,Going Ahead,Lost control,Had Been Drinking,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,Yes,<Null>,<Null>,<Null>,D22,19,Long Branch (19),13663


In [8]:
looper

,LATITUDE,LONGITUDE,INVAGE,WARDNUM,RDSFCOND,VISIBILITY,LIGHT,TRAFFCTL,VEHTYPE,DRIVACT,DRIVCOND,SPEEDING,AG_DRIV,ALCOHOL,DISABILITY,REDLIGHT
0,43.59,-79.54,20 to 24,3,Dry,Clear,Dark,No Control,"Automobile, Station Wagon",Driving Properly,Normal,<Null>,<Null>,<Null>,<Null>,<Null>
1,43.59,-79.53,unknown,3,Dry,Clear,Daylight,Traffic Signal,"Automobile, Station Wagon",<Null>,<Null>,<Null>,Yes,<Null>,<Null>,<Null>
2,43.59,-79.52,30 to 34,3,Dry,Clear,Daylight,No Control,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>
3,43.59,-79.51,20 to 24,3,Dry,Clear,Dark,No Control,"Automobile, Station Wagon",Driving Properly,Normal,<Null>,<Null>,<Null>,<Null>,<Null>
4,43.59,-79.50,40 to 44,3,Dry,Clear,Daylight,Traffic Signal,Pick Up Truck,Failed to Yield Right of Way,Inattentive,<Null>,Yes,<Null>,<Null>,<Null>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,43.84,-79.19,20 to 24,25,Dry,Clear,Dark,No Control,"Automobile, Station Wagon",Driving Properly,Normal,<Null>,Yes,<Null>,<Null>,<Null>
684,43.85,-79.19,45 to 49,25,Wet,Clear,Daylight,No Control,"Automobile, Station Wagon",Driving Properly,Normal,<Null>,<Null>,<Null>,<Null>,<Null>
685,43.85,-79.18,60 to 64,25,Wet,Clear,Daylight,No Control,Other,<Null>,<Null>,Yes,Yes,<Null>,<Null>,<Null>
686,43.85,-79.17,45 to 49,25,Dry,Clear,Daylight,No Control,Motorcycle,Driving Properly,Normal,<Null>,Yes,<Null>,<Null>,<Null>


In [9]:
looper.to_csv('KSI_Reg_Collapse.csv')